In [8]:
import pandas as pd
df=pd.read_excel("data.xlsx")
df[0:10]


,id,line,account,date_trans,date_entry,dc,amount,seed
0,1,1,18,2020-07-01,2020-07-04,C,-34278.37,0
1,1,2,6,2020-07-01,2020-07-04,D,26341.71,0
2,1,3,17,2020-07-01,2020-07-04,D,7936.66,0
3,2,1,16,2020-07-02,2020-07-04,C,-11525.70,0
4,2,2,1,2020-07-02,2020-07-04,D,11525.70,0
5,3,1,12,2020-07-04,2020-07-04,C,-19091.40,0
6,3,2,10,2020-07-04,2020-07-04,D,39214.51,0
7,4,1,19,2020-07-01,2020-07-05,C,-43888.82,0
8,4,2,18,2020-07-01,2020-07-05,D,45922.77,0
9,4,3,4,2020-07-01,2020-07-05,D,42045.58,0


In [9]:
field_to_check='amount'
list_to_check=df[field_to_check].to_list()
list_to_check=[int(round(x,2)*100) for x in list_to_check]
matched=[False for n in range(len(list_to_check))]
tuples=list(zip(df.index,list_to_check))
debs_pos=[x[0] for x in tuples if x[1] > 0]
debs_amt=[x[1] for x in tuples if x[1] > 0]
creds_pos=[x[0] for x in tuples if x[1] < 0]
creds_amt=[x[1] for x in tuples if x[1] < 0]
print(list_to_check[:10])
print(debs_amt[0:10])
print(creds_amt[0:10])

[-3427837, 2634171, 793666, -1152570, 1152570, -1909140, 3921451, -4388882, 4592277, 4204558]
[2634171, 793666, 1152570, 3921451, 4592277, 4204558, 2622169, 182330, 3176955, 508146]
[-3427837, -1152570, -1909140, -4388882, -3420676, -1453197, -3797430, -763749, -4039797, -4723325]


In [10]:
def check_match_list():
    running=0
    for n in match_list:
        tot=0
        for m in n:
            for v in m:
                tot=tot+df.iloc[v]['amount']
        if round(tot,1)==0:
            running=running+1
    print(running,len(match_list))

In [11]:
def find_possible_matches(v,list_amounts,ref="", list_refs=[]):
    matches=[]
    pos=[]
    ref=[]
    start=0
    while True:
        try:
            p=list_amounts.index(v,start)
            pos.append(p)
            start=p+1
        except Exception as e:
            break
    if len(pos)==1:
        return pos[0],pos, 1
    if len(pos)==0:
        return False, False, False
    if len(pos)>1:
        return pos[0], pos, len(pos)  
        #here we would develop something to choose, either least tolerance, or first/last 
        # or using regexp to select based on a potential reference field  
test=[1,3,200,100,1,3,200.01,200.001, 200.0001]
find_possible_matches(200,test) 

(2, [2], 1)

In [13]:
import traceback
match_list=[]
for i, v in enumerate(list_to_check):
    if v<0: #it is a credit, we search for debits that match
        try:      
            pos_deb, matched_pos, matched_count =find_possible_matches(-v, debs_amt) #debs_amt.index(-v)
            if pos_deb==False:
                continue    
            matched[debs_pos[pos_deb]]=True
            matched[i]=True
            match_list.append([[debs_pos[pos_deb]],[i]])
            debs_pos.pop(pos_deb)
            debs_amt.pop(pos_deb)        
            pointer=creds_pos.index(i)
            creds_pos.pop(pointer)
            creds_amt.pop(pointer)
        except Exception as e:
            if type(e).__name__ == 'ValueError':
                pass
            else:
                print(e)
                print(traceback.format_exc())
                break
    else:
        try:
            pos_cred, matched_pos, matched_count=find_possible_matches(-v, creds_amt) # creds_amt.index(-v)          
            if pos_cred==False:
                continue
            matched[i]=True
            matched[creds_pos[pos_cred]]=True
            match_list.append([[i],[creds_pos[pos_cred]]])         
            pointer=debs_pos.index(i)
            debs_pos.pop(pointer)
            debs_amt.pop(pointer)         
            creds_pos.pop(pos_cred)
            creds_amt.pop(pos_cred)        
        except Exception as e:
            if type(e).__name__ == 'ValueError':
                pass
            else:
                print(e)              
                print(traceback.format_exc())               
                break
match_list[0:4]

[[[4], [3]], [[192], [30]], [[193], [31]], [[194], [32]]]

In [14]:
check_match_list()

1174 1174


In [15]:
print(len(debs_pos))
print(len(creds_pos))
print(len(list_to_check),len(debs_pos)+len(creds_pos))

1362
1065
4754 2427


In [16]:
from itertools import combinations
deb_perm_list=list(combinations(debs_pos,2))
cred_perm_list=list(combinations(creds_pos,2))


In [17]:
deb_perm_list_sum=[]
for x in deb_perm_list:
    try:   
        deb_perm_list_sum.append(list_to_check[x[0]]+list_to_check[x[1]])
    except:
        print("Error with ", x)
        break
cred_perm_list_sum=[]
for x in cred_perm_list:
    try:   
        cred_perm_list_sum.append(list_to_check[x[0]]+list_to_check[x[1]])
    except:
        print("Error with ", x)
        break


In [18]:
for i, v in enumerate(list_to_check):
    if matched[i]==False:
        if v<0:
            try:
                #here we search the credit in the list of combinations of debits taken by 2, summed
                pos_deb, matched_pos, matched_count =find_possible_matches(-v,deb_perm_list_sum)
                if pos_deb==False:
                    continue
                    
                matched[i]=True
                matched[deb_perm_list[pos_deb][0]]=True
                matched[deb_perm_list[pos_deb][1]]=True             
                match_list.append([list(deb_perm_list[pos_deb]),[i]])
                
                #we remove the matched elements from the list of sums and their pointers 
                          
                debs_pointer1=debs_pos.index(deb_perm_list[pos_deb][0])
                debs_pos.pop(debs_pointer1)
                debs_amt.pop(debs_pointer1)
                
                debs_pointer2=debs_pos.index(deb_perm_list[pos_deb][1])
                debs_pos.pop(debs_pointer2)
                debs_amt.pop(debs_pointer2)
                
                creds_pointer=creds_pos.index(i)
                creds_pos.pop(creds_pointer)
                creds_amt.pop(creds_pointer)
                
                deb_perm_list_sum.pop(pos_deb)
                deb_perm_list.pop(pos_deb)
                
            except Exception as e:
                if type(e).__name__ == 'ValueError':
                    pass
                else:
                    print(e)
                    break
                
        else:
            try:
                pos_cred, matched_pos, matched_count =find_possible_matches(-v,cred_perm_list_sum)
                if pos_cred==False:
                    continue          
                matched[i]=True         
                matched[cred_perm_list[pos_cred][0]]=True
                matched[cred_perm_list[pos_cred][1]]=True           
                match_list.append([[i], list(cred_perm_list[pos_cred])])
                
                creds_pointer1=creds_pos.index(cred_perm_list[pos_cred][0])
                creds_pos.pop(creds_pointer1)
                creds_amt.pop(creds_pointer1)
                
                creds_pointer2=creds_pos.index(cred_perm_list[pos_cred][1])
                creds_pos.pop(creds_pointer2)
                creds_amt.pop(creds_pointer2)
                
                debs_pointer=debs_pos.index(i)
                debs_pos.pop(debs_pointer)
                debs_amt.pop(debs_pointer)
                
                cred_perm_list_sum.pop(pos_cred)
                cred_perm_list.pop(pos_cred)
                
                
                
            except Exception as e:
                if type(e).__name__ == 'ValueError':
                    pass
                else:
                    print(e)
                    break

In [19]:
match_list[-20:]

[[[3384, 3443], [4238]],
 [[4241], [1615, 2359]],
 [[4246], [4244, 4245]],
 [[4250], [3302, 4572]],
 [[4287], [708, 2028]],
 [[4324], [175, 4542]],
 [[216, 396], [4360]],
 [[2220, 3487], [4384]],
 [[4385], [272, 497]],
 [[4437], [1162, 1578]],
 [[4448], [351, 3431]],
 [[4459], [638, 1038]],
 [[4483], [2092, 4365]],
 [[4491], [4489, 4490]],
 [[4513], [2200, 3806]],
 [[4538], [927, 2494]],
 [[354, 1490], [4545]],
 [[4573], [840, 1850]],
 [[4591], [878, 1626]],
 [[4714], [4712, 4713]]]

In [20]:
print("% pending to reconcile: ",(len(debs_amt)+len(creds_amt))/len(list_to_check))

% pending to reconcile:  0.3901977282288599


In [22]:
check_match_list()

1408 1408


In [35]:
import numpy as np
matched_id=[np.nan for n in range(len(list_to_check))]
for i, t in enumerate(match_list):
    m=t[0]+t[1]
    for n in m:
        matched_id[n]=i+1
df1=pd.DataFrame(zip(matched, matched_id),columns=["matched","matched_id"])
df1.head()

,matched,matched_id
0,False,NaN
1,False,NaN
2,True,1375.0
3,True,1.0
4,True,1.0


In [36]:
df2=pd.merge(df,df1, left_index=True, right_index=True)

In [39]:
df2[df2.matched==True].sort_values("matched_id", ascending=False)[0:40]

,id,line,account,date_trans,date_entry,dc,amount,seed,matched,matched_id
4714,992,3,5,2021-07-02,2021-07-02,D,955.40,1,True,1408.0
4712,992,1,3,2021-07-02,2021-07-02,C,-292.94,2,True,1408.0
4713,992,2,3,2021-07-02,2021-07-02,C,-662.46,2,True,1408.0
878,190,2,15,2020-09-20,2020-09-20,C,-683.75,0,True,1407.0
4591,966,4,11,2021-06-27,2021-06-30,D,6355.92,0,True,1407.0
1626,343,1,10,2020-11-10,2020-11-14,C,-5672.17,0,True,1407.0
1850,390,2,13,2020-11-28,2020-11-28,C,-8490.01,0,True,1406.0
840,182,2,18,2020-09-16,2020-09-17,C,-18187.76,0,True,1406.0
4573,963,5,6,2021-06-28,2021-06-29,D,26677.76,0,True,1406.0
354,74,6,13,2020-08-02,2020-08-03,D,34783.52,0,True,1405.0


In [30]:
df2.to_excel("matched.xlsx")